#Amazon Product,Sales and Customer Analysis

In [13]:
import pymysql
import pandas as pd
import numpy as np

In [14]:
connection = pymysql.connect(host='localhost',user='root',password='HHLUADD@9586',database='endtoend')
connection

In [4]:
import warnings
warnings.filterwarnings('ignore')

query = 'select * from amazon'
data = pd.read_sql(query,connection)
data.head(2)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,timeofday,dayname,monthname
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1,Afternoon,Sat,Jan
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6,Morning,Fri,Mar


### Data Wrangling

In [5]:
data.isnull().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
timeofday                  0
dayname                    0
monthname                  0
dtype: int64

In [6]:
data.nunique()

Invoice ID                 1000
Branch                        3
City                          3
Customer type                 2
Gender                        2
Product line                  6
Unit price                  943
Quantity                     10
Tax 5%                      990
Total                       990
Date                         89
Time                        506
Payment                       3
cogs                        990
gross margin percentage       1
gross income                990
Rating                       61
timeofday                     3
dayname                       7
monthname                     3
dtype: int64

Feature Engineering = Generate some new columns

In [15]:
data.shape

(1000, 20)

### Q1: What is the count of distinct cities in the dataset?

In [8]:
import warnings
warnings.filterwarnings('ignore')

qry = 'select city ,count(city) as city_count  from amazon group by city'
ans = pd.read_sql(qry,connection)
ans



,city,city_count
0,Yangon,340
1,Naypyitaw,328
2,Mandalay,332


### Q2: For each branch, what is the corresponding city?

In [70]:
qry = 'select branch,city from amazon'
ans = pd.read_sql(qry,connection)
ans.head()

,branch,city
0,A,Yangon
1,C,Naypyitaw
2,A,Yangon
3,A,Yangon
4,A,Yangon


### Q3: What is the count of distinct product lines in the dataset?

In [10]:
qry = 'select `product line`,count(`product line`) as product_line_count from amazon group by `product line` order by product_line_count desc'
ans = pd.read_sql(qry,connection)
ans

,product line,product_line_count
0,Fashion accessories,178
1,Food and beverages,174
2,Electronic accessories,170
3,Sports and travel,166
4,Home and lifestyle,160
5,Health and beauty,152


### Q4: Which payment method occurs most frequently?

In [68]:
qry = 'select payment , count(payment) as payment_count from amazon group by payment order by payment_count  desc'
ans = pd.read_sql(qry,connection)
ans

,payment,payment_count
0,Ewallet,345
1,Cash,344
2,Credit card,311


### Q5: Which product line has the highest sales?

In [80]:
qry = 'select `product line` , count(`product line`) as sales_count from amazon group by `product line` order by sales_count desc limit 1'
ans = pd.read_sql(qry,connection)
ans

,product line,sales_count
0,Fashion accessories,178


### Q6: How much revenue is generated each month?

In [189]:
qry = 'select monthname , round(sum(`gross income`)) as revenue from amazon group by monthname '
ans = pd.read_sql(qry,connection)
ans

,monthname,revenue
0,Jan,5538.0
1,Mar,5212.0
2,Feb,4629.0


### Q7: In which month did the cost of goods sold reach its peak?

In [190]:
qry = 'select monthname , round(sum(cogs)) as cogs_ from amazon group by monthname order by cogs_ desc limit 1'
ans = pd.read_sql(qry,connection)
ans

,monthname,cogs_
0,Jan,110754.0


### Q8: Which product line generated the highest revenue?

In [191]:
qry = 'select `product line` , round(sum(total)) as highest_revenue from amazon group by `product line` order by highest_revenue desc limit 1'
ans = pd.read_sql(qry,connection)
ans

,product line,highest_revenue
0,Food and beverages,56145.0


### Q9: In which city was the highest revenue recorded?

In [192]:
qry = 'select city , round(sum(`gross income`)) as revenue from amazon group by city order by revenue desc limit 1 '
ans = pd.read_sql(qry,connection)
ans

,city,revenue
0,Naypyitaw,5265.0


### Q10: Which product line incurred the highest Value Added Tax?

In [193]:
qry = 'select `product line` , round(sum(`Tax 5%`)) as highest_value_tax from amazon group by `product line` order by highest_value_tax desc limit 1'
ans = pd.read_sql(qry,connection)
ans

,product line,highest_value_tax
0,Food and beverages,2674.0


### Q11: For each product line, add a column indicating "Good" if its sales are above average, otherwise "Bad."

In [188]:
qry = "SELECT `product line`, CASE WHEN quantity > 5.51 THEN 'Good' ELSE 'Bad' END AS sales_quality FROM amazon"
ans = pd.read_sql(qry,connection)
ans

,product line,sales_quality
0,Health and beauty,Good
1,Electronic accessories,Bad
2,Home and lifestyle,Good
3,Health and beauty,Good
4,Sports and travel,Good
...,...,...
995,Health and beauty,Bad
996,Home and lifestyle,Good
997,Food and beverages,Bad
998,Home and lifestyle,Bad


### Q12: Identify the branch that exceeded the average number of products sold.

In [194]:
qry = "SELECT Branch, round(avg(quantity),2) as product_sold FROM amazon group by branch"
ans = pd.read_sql(qry,connection)
ans

,Branch,product_sold
0,A,5.47
1,C,5.58
2,B,5.48


### Q13: Which product line is most frequently associated with each gender?

In [125]:
qry = "SELECT `product line`,COUNT(CASE WHEN gender = 'male' THEN 1 ELSE NULL END) AS male_count,COUNT(CASE WHEN gender = 'female' THEN 1 ELSE NULL END) AS female_count FROM amazon GROUP BY `product line`"
ans = pd.read_sql(qry,connection)
ans

,product line,male_count,female_count
0,Health and beauty,88,64
1,Electronic accessories,86,84
2,Food and beverages,84,90
3,Fashion accessories,82,96
4,Home and lifestyle,81,79
5,Sports and travel,78,88


### Q14: Calculate the average rating for each product line.

In [11]:
qry = "SELECT `product line`, round(avg(rating),2) as avg_rating FROM amazon group by `product line` order by avg_rating desc"
ans = pd.read_sql(qry,connection)
ans

,product line,avg_rating
0,Food and beverages,7.11
1,Fashion accessories,7.03
2,Health and beauty,7.00
3,Electronic accessories,6.92
4,Sports and travel,6.92
5,Home and lifestyle,6.84


### Q15: Count the sales occurrences for each time of day on every weekday.

In [129]:
qry = "SELECT dayname, count(quantity) as sales_count FROM amazon group by dayname"
ans = pd.read_sql(qry,connection)
ans

,dayname,sales_count
0,Sat,164
1,Fri,139
2,Sun,133
3,Mon,125
4,Thu,138
5,Wed,143
6,Tue,158


### Q16: Identify the customer type contributing the highest revenue.

In [197]:
qry = "SELECT `customer type`, round(sum(`gross income`)) as revenue from amazon group by `customer type`"
ans = pd.read_sql(qry,connection)
ans

,customer type,revenue
0,Member,7820.0
1,Normal,7559.0


### Q17: Determine the city with the highest VAT percentage.

In [198]:
qry = "SELECT city, round(sum(`tax 5%`)) as VAT from amazon group by city order by VAT desc"
ans = pd.read_sql(qry,connection)
ans.head(1)

,city,VAT
0,Naypyitaw,5265.0


### Q18: Identify the customer type with the highest VAT payments.

In [199]:
qry = "SELECT `customer type`, round(sum(`tax 5%`)) as VAT from amazon group by `customer type` order by VAT desc"
ans = pd.read_sql(qry,connection)
ans.head(1)

,customer type,VAT
0,Member,7820.0


### Q19: What is the count of distinct customer types in the dataset?

In [152]:
qry = 'select `customer type` , count(`customer type`) as distinct_customer  from amazon group by `customer type`'
ans = pd.read_sql(qry,connection)
ans.head()

,customer type,distinct_customer
0,Member,501
1,Normal,499


### Q20: What is the count of distinct payment methods in the dataset?

In [151]:
qry = 'select payment  , count(payment) as distinct_payment  from amazon group by payment '
ans = pd.read_sql(qry,connection)
ans.head()

,payment,distinct_payment
0,Ewallet,345
1,Cash,344
2,Credit card,311


### Q21: Which customer type occurs most frequently?

In [153]:
qry = 'select `customer type` , count(`customer type`) as customer_count  from amazon group by `customer type`'
ans = pd.read_sql(qry,connection)
ans.head()

,customer type,customer_count
0,Member,501
1,Normal,499


### Q22: Identify the customer type with the highest purchase frequency.

In [16]:
qry = 'select `customer type` , round(sum(total)) as total_cost  from amazon group by `customer type` order by total_cost desc'
ans = pd.read_sql(qry,connection)
ans.head()

,customer type,total_cost
0,Member,164223.0
1,Normal,158743.0


### Q23: Determine the predominant gender among customers

In [167]:
qry = "select `customer type`,count(case when gender = 'male' then 1 else null end) as male_count,count(case when gender = 'female' then 1 else null end) as female_count from amazon group by `customer type`"

In [168]:
ans = pd.read_sql(qry,connection)
ans.head()

,customer type,male_count,female_count
0,Member,240,261
1,Normal,259,240


### Q24: Examine the distribution of genders within each branch.

In [174]:
qry = "select branch, count(case when gender = 'male' then 1 else null end) as male_count,count(case when gender = 'female' then 1 else null end) as female_count from amazon group by branch"

In [175]:
ans = pd.read_sql(qry,connection)
ans.head()

,branch,male_count,female_count
0,A,179,161
1,C,150,178
2,B,170,162


### Q25: Identify the time of day when customers provide the most ratings.

In [20]:
qry = 'select distinct(time),avg(rating) as ratings from amazon group by time order by ratings desc'
ans = pd.read_sql(qry,connection)
ans.head()

,time,ratings
0,19:18:00,10.0
1,14:45:00,9.9
2,17:48:00,9.9
3,15:18:00,9.9
4,15:39:00,9.8


### Q26: Determine the time of day with the highest customer ratings for each branch.

In [184]:
qry = 'select time,branch,avg(rating) as ratings from amazon group by branch,time order by ratings desc'
ans = pd.read_sql(qry,connection)
ans.head()

,time,branch,ratings
0,17:47:00,C,10.0
1,19:18:00,B,10.0
2,11:15:00,A,9.9
3,16:44:00,A,9.9
4,18:23:00,C,9.9


### Q27: Identify the day of the week with the highest average ratings.

In [200]:
qry = 'select dayname,round(avg(rating),2) as ratings from amazon group by dayname order by ratings desc'
ans = pd.read_sql(qry,connection)
ans.head()

,dayname,ratings
0,Mon,7.15
1,Fri,7.08
2,Sun,7.01
3,Tue,7.00
4,Sat,6.90


### Q28: Determine the day of the week with the highest average ratings for each branch.

 

In [201]:
qry = 'select dayname,branch,round(avg(rating),2) as ratings from amazon group by dayname,branch order by ratings desc'
ans = pd.read_sql(qry,connection)
ans.head()

,dayname,branch,ratings
0,Mon,B,7.34
1,Fri,A,7.31
2,Fri,C,7.28
3,Sat,C,7.23
4,Mon,A,7.10
